In [ ]:
#!pip install scikit-learn-intelex
#!pip install intel-extension-for-transformers
#!pip install -U sentence-transformers
#!pip install keybert

In [ ]:
## We are using intel_extension_for_pytorch Model Optimization and sklearnx for faster Sklearn Processing

In [5]:
!pip install intel_extension_for_pytorch

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import torch
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/LaBSE')

In [ ]:
import intel_extension_for_pytorch as ipex
model = ipex.optimize(model, dtype=torch.bfloat16)

In [7]:
from keybert import KeyBERT
kw_model = KeyBERT(model=model)

In [8]:
from datasets import load_from_disk

all_data = load_from_disk('clean_resume_data')

In [9]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [10]:
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
import re
import pandas as pd
import emoji
def get_re_compiler_list_words(words_list):
    compiled_str = r''
    for i, word in enumerate(words_list):
        if i == len(words_list) - 1:
            compiled_str = compiled_str + r'\b' + word + r'\b'
        else:
            compiled_str = compiled_str + r'\b' + word + r'\b | '
    r = re.compile(compiled_str, flags=re.I | re.X)

    return r


def get_compiler_removed_stopwords():
    stopword_list = ['follow', 'channel', 'news', 'source', 'breaking', 'tv', 'watch', 'video', 'support', 'subscribe',
                     'share', 'link', 'comment', 'download', 'free', 'post', 'click', 'online', 'tube', 'call', 'plz',
                     'donate', 'help', 'shared', 'pls', 'sms', 'likes', 'copy', 'following', 'retweet', 'website',
                     'comments', 'notification', 'updates', 'play', 'pay', 'msg', 'quotes', 'block', 'posts', 'rekoooo',
                     'sent', 'from', 'here', 'android']
    r = get_re_compiler_list_words(stopword_list)

    return r

stop_words_regex = get_compiler_removed_stopwords()
url_pattern = re.compile(r'(https?://[^\s]+)')
text_compile1 = re.compile("&#39;")
text_compile2 = re.compile('[-_[\]{}~\':;"’‘()–\n\r<>@&*+!?•°.,\\\/%=^$|#“”]+')
ptrn = re.compile('[^a-z ]')


def get_emoji_regexp():
    # Sort emoji by length to make sure multi-character emojis are
    # matched first
    emojis = sorted(emoji.EMOJI_DATA, key=len, reverse=True)
    pattern = u'(' + u'|'.join(re.escape(u) for u in emojis) + u')'
    return re.compile(pattern)

def remove_emoji(text):
    outp = get_emoji_regexp().sub(u'', str(text))
    return outp

def remove_mentions_and_hashtag(text):
    
    outp = " ".join(filter(lambda x: x[0] not in ['#', '@'], text.split()))
    return outp

def remove_unnecessary_text(example):
    text = example['Resume_str']
    outp = url_pattern.sub('', text)
    outp = stop_words_regex.sub('', outp)
    outp = remove_mentions_and_hashtag(remove_emoji(outp))
    outp = text_compile1.sub('', outp)
    outp = text_compile2.sub('', outp)
    
    example['Resume_str'] = outp

    return example

In [12]:
def remove_unnecessary_text2(example):
    text = example
    outp = url_pattern.sub('', text)
    outp = stop_words_regex.sub('', outp)
    outp = remove_mentions_and_hashtag(remove_emoji(outp))
    outp = text_compile1.sub('', outp)
    outp = text_compile2.sub('', outp)

    return outp

In [13]:
def sent_check(example):
    
    example['embed'] = model.encode(example['Resume_str'])
    
    return example

In [14]:
def keyword_extraction(example):
    
    example['keywords'] = [i[0] for i in kw_model.extract_keywords(example['Resume_str'], keyphrase_ngram_range=(1, 1), stop_words='english',
                              use_maxsum=True, nr_candidates=20, top_n=10)]
    
    return example

In [15]:
def label_change(example):
    
    example['Category'] = example['Category'].lower()
    
    return example

In [ ]:
all_data = all_data.map(remove_unnecessary_text)
all_data = all_data.map(sent_check, batched=True)

In [ ]:
all_data = all_data.map(keyword_extraction)

  0%|          | 0/3446 [00:00<?, ?ex/s]

In [ ]:
all_data = all_data.map(label_change)

In [86]:
def sim_match(example, embed):
    
    example['cos_sim'] = cosine_similarity([embed, example['embed']])[0][1]
    
    tmp = example['Resume_str'].lower().split(' ')
    example['key_match'] = len([i for i in jd_key if i in tmp])
    
    return example

In [96]:
def similar_profiles(jd, all_data):
    
    try:
        all_data = all_data.remove_columns(['cos_sim', 'key_match'])
    except:
        pass
    
    jd_clean = remove_unnecessary_text2(jd)
    jd_embed = model.encode(jd_clean)
    jd_key = [i[0] for i in kw_model.extract_keywords(jd_clean, keyphrase_ngram_range=(1, 1), stop_words='english',
                              use_maxsum=True, nr_candidates=20, top_n=10)]
    
    all_data = all_data.map(sim_match, fn_kwargs={"embed":jd_embed})
    res_df = all_data.filter(lambda x: x['cos_sim'] > 0.72 and x['key_match'] > 5).to_pandas()
    
    return res_df    

In [97]:
jd_hr = """The HR Specialist will be responsible for a variety of HR functions including but not limited to recruiting, onboarding, benefits administration, employee relations, and compliance. The successful candidate will work closely with management to ensure compliance with all applicable employment laws and regulations, and will help create a positive work environment that fosters employee engagement and retention.

Key Responsibilities:

Manage the recruitment process from job postings to offer letters, including candidate screening, interviewing, and reference checks
Assist in the onboarding process for new hires, including conducting orientation sessions and ensuring all necessary paperwork is completed
Administer employee benefit programs, including health insurance, retirement plans, and other benefits
Serve as a point of contact for employee questions and concerns related to HR policies and procedures
Work closely with management to identify and address employee relations issues in a timely and effective manner
Ensure compliance with all federal, state, and local employment laws and regulations
Maintain employee records and ensure accuracy of HR systems and data
"""

In [100]:
results = similar_profiles(jd_hr, all_data)

results = results.sort_values(by='cos_sim', ascending=False)
results

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2023-02-16 04:35:11,167 - datasets.arrow_dataset - WARNING - Loading cached processed dataset at /home/u185998/clean_resume_data/cache-865bd72de8c1d155.arrow
2023-02-16 04:35:11,170 - datasets.arrow_dataset - WARNING - Loading cached processed dataset at /home/u185998/clean_resume_data/cache-eae7a196d0d1e7ba.arrow


,Resume_str,Category,embed,cos_sim,key_match
11,HR COORDINATOR Summary Certified Human Resourc...,HR,"[-0.02826775, -0.039918706, -0.013609606, -0.0...",0.854225,7
17,HR MANAGER Summary Human Resources Professiona...,HR,"[-0.04598558, -0.040164113, -0.009126835, -0.0...",0.850612,7
14,HR ASSISTANT Highlights Interviewing expertise...,HR,"[-0.043813698, -0.02912577, -0.017574316, -0.0...",0.850336,7
12,HR GENERALIST Summary Energetic Bilingual Huma...,HR,"[-0.05544784, -0.026709273, -0.012451196, -0.0...",0.846114,8
62,CONSULTANT Summary Human Resource Professional...,CONSULTANT,"[-0.057123452, -0.036554333, -0.023314979, 0.0...",0.819709,6
...,...,...,...,...,...
61,BUSINESS ACCOUNT LEAD Executive Profile Strong...,BPO,"[-0.03645325, -0.014745852, 0.028635135, -0.00...",0.727530,6
37,HR SAFETY MANAGER Summary Human Resources Man...,HR,"[-0.03956906, -0.021043437, -0.027549429, 0.01...",0.726188,6
69,BRANCH MANAGER Summary As an experienced banki...,BANKING,"[-0.034344185, -0.025618272, -0.035721105, -0....",0.723338,7
59,CLINICAL STUDY COORDINATOR II Professional Bac...,FITNESS,"[-0.020330684, -0.048541896, 0.030323666, -0.0...",0.722900,6


In [91]:
jd_info = """"Responsibilities:

Collaborate with cross-functional teams to design, develop, and deploy high-quality software solutions
Provide technical leadership and guidance to junior developers
Participate in code reviews and ensure adherence to best practices and coding standards
Work closely with product managers to define and prioritize product requirements and develop user stories
Build and maintain scalable and reliable software systems and architecture
Troubleshoot and debug production issues and implement corrective actions
Continuously improve and optimize software performance and scalability
Research and evaluate new technologies and tools to improve development efficiency and productivity
Contribute to the development and implementation of software engineering best practices and processes
"""

In [99]:
results = similar_profiles(jd_info, all_data)

results = results.sort_values(by='cos_sim', ascending=False)
results

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2023-02-16 04:34:36,653 - datasets.arrow_dataset - WARNING - Loading cached processed dataset at /home/u185998/clean_resume_data/cache-357bbdbbaa86935c.arrow


  0%|          | 0/4 [00:00<?, ?ba/s]

,Resume_str,Category,embed,cos_sim,key_match
0,HR MANAGER Summary To obtain a position that o...,HR,"[-0.03701455, -0.019948926, -0.023139175, -0.0...",0.720511,6


In [103]:
job_ds = """Experience: 5+ years of experience in aviation engineering

Responsibilities:

Design and develop aircraft and aircraft systems, such as engines, avionics, flight controls, and landing gear
Conduct research and development on new and emerging aviation technologies and materials
Collaborate with cross-functional teams, including aerodynamics, structures, and systems engineering, to ensure aircraft designs meet performance, safety, and regulatory requirements
Analyze and evaluate aircraft performance, including speed, range, fuel efficiency, and payload
Use specialized software and tools to simulate and model aircraft and aircraft systems
Perform structural analysis and testing to ensure aircraft components and systems meet safety and reliability standards
Develop and maintain technical documentation, including engineering drawings, specifications, and reports
Collaborate with manufacturing and production teams to ensure designs are producible and efficient
Participate in certification and testing of aircraft and aircraft systems
Maintain knowledge of industry standards, regulations, and best practices"""

In [104]:
results = similar_profiles(job_ds, all_data)

results = results.sort_values(by='cos_sim', ascending=False)
results

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3446 [00:00<?, ?ex/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

,Resume_str,Category,embed,cos_sim,key_match
0,AVIATION SUPPLY SPECIALIST Summary Organized p...,AVIATION,"[-0.053045448, 0.026786014, -0.027348204, -0.0...",0.726557,6


In [108]:
job_chef = """5+ years of experience in culinary arts

Responsibilities:

Plan and prepare high-quality and delicious meals that meet customer preferences and dietary requirements
Develop and modify menus, recipes, and ingredients to meet changing customer demands
Estimate food requirements, order food supplies, and maintain inventory levels
Supervise and coordinate kitchen staff to ensure smooth operations, including food preparation, cooking, and cleaning
Train and mentor kitchen staff members to ensure quality and consistency in food preparation and cooking
Ensure compliance with food safety and sanitation standards, including HACCP and ServSafe guidelines
Develop and implement cost-saving strategies to minimize food waste and maximize profitability
Collaborate with restaurant management and front-of-house staff to ensure customer satisfaction and efficient operations
Maintain knowledge of current culinary trends and techniques
Maintain a clean and organized kitchen, including equipment maintenance and repair"""

In [109]:
results = similar_profiles(job_chef, all_data)

results = results.sort_values(by='cos_sim', ascending=False)
results

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3446 [00:00<?, ?ex/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

,Resume_str,Category,embed,cos_sim,key_match
3,Summary I am a kitchen manager with six years ...,CHEF,"[-0.05047101, 0.008844882, -0.057153843, -0.04...",0.730248,6
2,DEPUTY PRINCIPAL Executive Profile Resultsorie...,AUTOMOBILE,"[-0.032646265, -0.04653155, -0.019676032, -0.0...",0.724661,6
0,HR ASSISTANT Highlights Interviewing expertise...,HR,"[-0.043813698, -0.02912577, -0.017574316, -0.0...",0.724054,7
1,CUSTOMER CARE REPRESENTATIVE Professional Summ...,FITNESS,"[-0.027356727, -0.020181151, -0.006641451, -0....",0.723831,6
